# 🧠 Agentic RAG with Shared Memory using TiDB Serverless + PyTiDB
This notebook demonstrates how RAG agents share memory in TiDB Serverless using `pytidb`, vector embedding with OpenAI, and memory retrieval with vector search.

In [19]:
%pip install -q pytidb "pytidb[models]" openai python-dotenv pandas

Note: you may need to restart the kernel to use updated packages.


In [20]:
import dotenv
dotenv.load_dotenv()

True

In [21]:
from pytidb import TiDBClient
from openai import OpenAI
from datetime import datetime

import os

db = TiDBClient.connect(
    host=os.getenv("TIDB_HOST"),
    username=os.getenv("TIDB_USERNAME"),
    password=os.getenv("TIDB_PASSWORD"),
    database=os.getenv("TIDB_DATABASE"),
    port=int(os.getenv("TIDB_PORT"))
)


In [22]:
# Create table for memory store with vector embedding
from pytidb.schema import TableModel, Field
from pytidb.embeddings import EmbeddingFunction
dotenv.load_dotenv()
# Define your embedding model.

text_embed = EmbeddingFunction("openai/text-embedding-3-small")


class Memory(TableModel, table=True):
    __tablename__ = "memory"
    __table_args__ = {"extend_existing": True}

    id: int = Field(primary_key=True)
    agent_id: str = Field()
    memory_type: str = Field()
    content: str = Field()
    embedding: list[float] = text_embed.VectorField(
        source_field="content"
    )  # 👈 Define the vector field.
    tags: str = Field()
    visibility: str = Field()
    expires_at: datetime = Field()
    created_at: datetime = Field(default_factory=datetime.now)

table = db.create_table(schema=Memory)

/Users/xin/.pyenv/versions/3.12.4/lib/python3.12/site-packages/sqlmodel/main.py:641: SAWarning: This declarative base already contains a class with the same class name and module name as __main__.Memory, and will be replaced in the string-lookup table.
  DeclarativeMeta.__init__(cls, classname, bases, dict_, **kw)


In [23]:
from datetime import datetime, timedelta
table.truncate()

# Create the memory record — DO NOT assign `embedding` manually
memory = Memory(
    id=1,
    agent_id="agent_a",
    memory_type="short",
    content="Refund processed for failed transaction.",
    tags='{"topic": "refund"}',
    visibility="team",
    expires_at=datetime.now() + timedelta(days=2),
    created_at=datetime.now(),
)

# This will auto-generate the embedding using `text_embed.VectorField(...)`
table.insert(memory)

print("✅ Memory stored with ID:", memory.id)


✅ Memory stored with ID: 1


In [24]:
# Retrieve shared memory using vector similarity
query = "How to help a user with payment refund?"

res = (
    table.search(query)
    .filter({"visibility": "team"})  # Just one value first to avoid SQL error
    .limit(3)
    .to_pandas()
)

print("\n📚 Retrieved shared memory:")
for _, row in res.iterrows():
    print("-", row["content"])



📚 Retrieved shared memory:
- Refund processed for failed transaction.
